## Exercise Solution: Parallelization with Distributed

In [1]:
using Distributed


In [2]:

# Function to find prime numbers in a range using @distributed
function find_primes(start, stop)
    @everywhere function is_prime(n)
        if n <= 1
            return false
        end

        for i = 2:isqrt(n)
            if n % i == 0
                return false
            end
        end

        return true
    end

    results = @distributed (vcat) for n = start:stop
        if is_prime(n)
            [n]
        else
            Int[]
        end
    end

    return results
end



find_primes (generic function with 1 method)

In [3]:
# Set up the parallel workers
addprocs(4)

# Define the range to search for primes
start = 1
stop = 1000

# Perform the parallel prime number search
@time result = find_primes(start, stop)

# Print the result
println(result)

  0.757126 seconds (649.81 k allocations: 44.054 MiB, 0.57% gc time, 44.36% compilation time)
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]


# Q: Is there a more elegant parallelized solution for the $\pi$ approximation?

In [5]:
"""
    calculate_pi_parallel_direct(total_points::Int) -> Float64

Calculate an estimate of π using the Monte Carlo method, in parallel, by directly distributing the random point generation across workers.

Instead of assigning a fixed number of points to each worker, this function parallelizes the task of generating and checking each point. Each iteration of the internal loop can be run on any available worker, which allows for improved load balancing.

# Arguments
- `total_points::Int`: The total number of random points to use for the estimation.

# Returns
- `Float64`: An estimate of π.
"""
function calculate_pi_parallel_direct(total_points::Int)
    # Use @distributed for parallel reduction with a (+) operator, iterating over each point
    inside_total = @distributed (+) for i = 1:total_points
        x = rand()
        y = rand()
        (x^2 + y^2) <= 1.0 ? 1 : 0
    end
    
    # Calculate pi using the aggregated result
    return 4 * inside_total / total_points
end

calculate_pi_parallel_direct

In [6]:
calculate_pi_parallel_direct(Int(1e10))

3.1415922792

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2024 [Point 8 GmbH](https://point-8.de)_